In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("airbnb")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
listings_amsterdam_file_path = 'gs://assignment2_airbnb/airbnb_data/Amsterdam/listings_a.csv' 
listings_rotterdam_file_path = 'gs://assignment2_airbnb/airbnb_data/Rotterdam/listings_r.csv' 
listings_denhaag_file_path = 'gs://assignment2_airbnb/airbnb_data/DenHaag/listings_dh.csv' 
reviews_amsterdam_file_path = 'gs://assignment2_airbnb/airbnb_data/Amsterdam/reviews.csv' 
reviews_rotterdam_file_path = 'gs://assignment2_airbnb/airbnb_data/Rotterdam/reviews.csv' 
reviews_denhaag_file_path = 'gs://assignment2_airbnb/airbnb_data/DenHaag/reviews.csv' 


df_listings_amsterdam = spark.read.format("csv").option("header", "true") \
       .load(listings_amsterdam_file_path)
df_listings_rotterdam = spark.read.format("csv").option("header", "true") \
       .load(listings_rotterdam_file_path)
df_listings_denhaag = spark.read.format("csv").option("header", "true") \
       .load(listings_denhaag_file_path)
df_reviews_amsterdam = spark.read.format("csv").option("header", "true") \
       .load(reviews_amsterdam_file_path)
df_reviews_rotterdam = spark.read.format("csv").option("header", "true") \
       .load(reviews_rotterdam_file_path)
df_reviews_denhaag = spark.read.format("csv").option("header", "true") \
       .load(reviews_denhaag_file_path)



In [2]:
from pyspark.sql.functions import lit
df_listings_amsterdam=df_listings_amsterdam.withColumn("City", lit('Amsterdam'))
df_listings_rotterdam=df_listings_rotterdam.withColumn("City", lit('Rotterdam'))
df_listings_denhaag=df_listings_denhaag.withColumn("City", lit('Den Haag'))

In [3]:
df_listings_unfiltered=df_listings_amsterdam.union(df_listings_rotterdam.union(df_listings_denhaag))

In [4]:
df_listings=df_listings_unfiltered.select(['id','name','price','review_scores_rating'])
df_listings.show(50)

+------+--------------------+---------+--------------------+
|    id|                name|    price|review_scores_rating|
+------+--------------------+---------+--------------------+
|  2818|Quiet Garden View...|   $49.00|                4.89|
| 20168|Studio with priva...|  $106.00|                4.44|
| 27886|Romantic, stylish...|  $136.00|                4.94|
| 28871|Comfortable doubl...|   $75.00|                4.88|
| 29051|Comfortable singl...|   $55.00|                4.79|
| 44391|Quiet 2-bedroom A...|  $240.00|                4.72|
| 49552|Multatuli Luxury ...|  $245.00|                4.92|
| 50523|B & B de 9 Straat...|  $124.00|                4.87|
| 55709|Bright Loft in Ce...|  $250.00|                4.95|
| 55868|Apartment near Mu...|  $149.00|                4.86|
| 62015|Charming, beautif...|  $120.00|                4.91|
| 80635|TOP LOCATED Canal...|   $82.00|                4.77|
| 82482|The Backroom - Ce...|  $119.00|                 4.5|
| 91535|"Privat room and

In [5]:
df_listing_city=df_listings_unfiltered.select(['id','city'])

In [6]:
df_owner_listings=df_listings_unfiltered.select(['id','host_id'])

In [7]:
df_owner=df_listings_unfiltered.select(['host_id','host_name'])
df_owner=df_owner.dropDuplicates(['host_id'])

In [8]:
from pyspark.sql.functions import regexp_replace
#remove the dollar sign from the price column and convert it to float
df_listings = df_listings.withColumn("price", regexp_replace("price", "\$", ""))
df_listings = df_listings.withColumn("price", regexp_replace("price", "\,", ""))
df_listings = df_listings.withColumn("price", df_listings["price"].cast("float"))
#drop the wors where price is zero
df_listings = df_listings.filter(df_listings.price > 0)

In [9]:
#order the data by price
df_listings_ordered = df_listings.orderBy("price", ascending=True)

In [10]:

df_listings.createOrReplaceTempView("listings")
df_listing_city.createOrReplaceTempView("listing_city")
df_owner.createOrReplaceTempView("owner")
df_owner_listings.createOrReplaceTempView("owner_listings")

In [11]:
df_listings.show()

+------+--------------------+-----+--------------------+
|    id|                name|price|review_scores_rating|
+------+--------------------+-----+--------------------+
|  2818|Quiet Garden View...| 49.0|                4.89|
| 20168|Studio with priva...|106.0|                4.44|
| 27886|Romantic, stylish...|136.0|                4.94|
| 28871|Comfortable doubl...| 75.0|                4.88|
| 29051|Comfortable singl...| 55.0|                4.79|
| 44391|Quiet 2-bedroom A...|240.0|                4.72|
| 49552|Multatuli Luxury ...|245.0|                4.92|
| 50523|B & B de 9 Straat...|124.0|                4.87|
| 55709|Bright Loft in Ce...|250.0|                4.95|
| 55868|Apartment near Mu...|149.0|                4.86|
| 62015|Charming, beautif...|120.0|                4.91|
| 80635|TOP LOCATED Canal...| 82.0|                4.77|
| 82482|The Backroom - Ce...|119.0|                 4.5|
| 91535|"Privat room and ...|165.0|                 4.8|
| 97221|Beautiful and spa...| 6

In [12]:
min_nr_listings = 5

sql_query_highest_ratings = f"""SELECT
    owner.host_id as owner_id,
    round(avg(review_scores_rating), 2) as avg_rating,
    count(*) as num_listings
FROM
    listings, owner, owner_listings
WHERE
    listings.id = owner_listings.id
    and owner.host_id = owner_listings.host_id
GROUP BY
    owner.host_id
HAVING count(*) > {min_nr_listings}
ORDER BY
    avg_rating desc"""

highest_ratings = spark.sql(sql_query_highest_ratings)

print(f"Found {highest_ratings.count()} listings with at least {min_nr_listings} listings")


Found 81 listings with at least 5 listings


In [13]:
highest_ratings.toPandas().to_csv("gs://assignment2_airbnb/Results/highest_ratings.csv")

In [14]:
# Make a query to the Spark SQL engine
# Check which owners have at least one listing in each of the 3 cities

# Table names: listings, listing_city, owner, owner_listings, renter, review
# listings(id, name, price, review_scores_rating)
# listing_city(id, city)
# owner(host_id, host_name)
# owner_listings(id, host_id)
# renter(renter_id, name)
# review(renter_id, listing_id, review)

# Take owners that have listings in all of Amsterdam, Rotterdam and Den Haag

nr_cities = 2

query_owners_in_at_least_2_cities = f"""
SELECT
    owner.host_id,
    owner.host_name,
    count(distinct listing_city.city) as num_cities
FROM
    owner, owner_listings, listing_city
WHERE
    owner.host_id = owner_listings.host_id
    and owner_listings.id = listing_city.id
GROUP BY
    owner.host_id, owner.host_name
HAVING
    count(distinct listing_city.city) >= {nr_cities}

"""

results_owners_in_at_least_2_cities = spark.sql(query_owners_in_at_least_2_cities)

print(f"Found {results_owners_in_at_least_2_cities.count()} owners with listings in at least {nr_cities} cities")
results_owners_in_at_least_2_cities.show()

Found 8 owners with listings in at least 2 cities
+---------+----------------+----------+
|  host_id|       host_name|num_cities|
+---------+----------------+----------+
| 10239880|           Lucas|         2|
|115324475|             Lin|         2|
|121985032| Tess - BELVILLA|         2|
|128826790| Natasja & Mylan|         2|
|177701530| Hosted By Wendy|         2|
| 19894111|Arjen & Nathalie|         2|
| 20465009|          Alette|         2|
|244520390|           Gunni|         2|
+---------+----------------+----------+



In [15]:
results_owners_in_at_least_2_cities.toPandas().to_csv("gs://assignment2_airbnb/Results/owners_two_cities.csv")

In [16]:
# Table names: listings, listing_city, owner, owner_listings, renter, review
# listings(id, name, price, review_scores_rating)
# listing_city(id, city)
# owner(host_id, host_name)
# owner_listings(id, host_id)
# renter(renter_id, name)
# review(renter_id, listing_id, review)

min_listings = 10

# Find owners with at least 10 listings
query_owners_with_at_least_10_listings = """
SELECT
    owner.host_id as owner_id,
    owner.host_name as owner_name,
    COUNT(*) as num_listings,
    ROUND(AVG(listings.price), 2) as avg_price,
    ROUND(AVG(listings.review_scores_rating), 1) as avg_rating
FROM
    listings, owner, owner_listings
WHERE
    listings.id = owner_listings.id
    and owner.host_id = owner_listings.host_id
GROUP BY
    owner.host_id, owner.host_name
HAVING COUNT(*) > 10
ORDER BY
    num_listings DESC
"""

results_owners_10_listings = spark.sql(query_owners_with_at_least_10_listings)
print(f"Found {results_owners_10_listings.count()} owners with at least {min_listings} listings")

Found 24 owners with at least 10 listings


In [17]:
results_owners_10_listings.toPandas().to_csv("gs://assignment2_airbnb/Results/owners_10_listings.csv")

In [39]:
#select the 3 owners with the most listings per city
sql_query_most_listings_ams = f"""SELECT
    owner.host_id as owner_id,
    owner.host_name as owner_name,
    listing_city.city as city,
    count(*) as num_listings
FROM
    listings, owner, owner_listings, listing_city
WHERE
    listings.id = owner_listings.id
    and owner.host_id = owner_listings.host_id
    and listings.id = listing_city.id
    and city='Amsterdam'
GROUP BY
    owner.host_id, owner.host_name, listing_city.city
ORDER BY
    listing_city.city, num_listings desc
Limit 5    
"""

most_listings_ams = spark.sql(sql_query_most_listings)

+---------+---------------+---------+------------+
| owner_id|     owner_name|     city|num_listings|
+---------+---------------+---------+------------+
| 14574533|Hotel Not Hotel|Amsterdam|          21|
|138369331|          Peter|Amsterdam|          20|
|203731852|   SWEETS Hotel|Amsterdam|          20|
| 67005410|      Feliciano|Amsterdam|          17|
|241644101|     Wittenberg|Amsterdam|          14|
+---------+---------------+---------+------------+



In [43]:
sql_query_most_listings_dh = f"""SELECT
    owner.host_id as owner_id,
    owner.host_name as owner_name,
    listing_city.city as city,
    count(*) as num_listings
FROM
    listings, owner, owner_listings, listing_city
WHERE
    listings.id = owner_listings.id
    and owner.host_id = owner_listings.host_id
    and listings.id = listing_city.id
    and city='Den Haag'
GROUP BY
    owner.host_id, owner.host_name, listing_city.city
ORDER BY
    listing_city.city, num_listings desc
Limit 5    
"""

most_listings_dh = spark.sql(sql_query_most_listings_dh)
most_listings_dh.show()

+---------+----------+--------+------------+
| owner_id|owner_name|    city|num_listings|
+---------+----------+--------+------------+
|  5285926| Diederick|Den Haag|          29|
|  9165668|    Robert|Den Haag|          23|
|   300966|     Elwin|Den Haag|          20|
|244520390|     Gunni|Den Haag|          18|
| 10239880|     Lucas|Den Haag|          17|
+---------+----------+--------+------------+



In [44]:
sql_query_most_listings_r = f"""SELECT
    owner.host_id as owner_id,
    owner.host_name as owner_name,
    listing_city.city as city,
    count(*) as num_listings
FROM
    listings, owner, owner_listings, listing_city
WHERE
    listings.id = owner_listings.id
    and owner.host_id = owner_listings.host_id
    and listings.id = listing_city.id
    and city='Rotterdam'
GROUP BY
    owner.host_id, owner.host_name, listing_city.city
ORDER BY
    listing_city.city, num_listings desc
Limit 5    
"""

most_listings_r = spark.sql(sql_query_most_listings_r)
most_listings_r.show()

+---------+----------+---------+------------+
| owner_id|owner_name|     city|num_listings|
+---------+----------+---------+------------+
|448782489|   Martijn|Rotterdam|          36|
|432320567|    Tijmen|Rotterdam|          15|
|244141635|   Culture|Rotterdam|          12|
|436284753|Wikkelboat|Rotterdam|          11|
|302893992|   Michiel|Rotterdam|          10|
+---------+----------+---------+------------+



In [46]:
most_listings_per_city=most_listings_ams.union(most_listings_r.union(most_listings_dh))
most_listings_per_city.show(15)

+---------+---------------+---------+------------+
| owner_id|     owner_name|     city|num_listings|
+---------+---------------+---------+------------+
| 14574533|Hotel Not Hotel|Amsterdam|          21|
|138369331|          Peter|Amsterdam|          20|
|203731852|   SWEETS Hotel|Amsterdam|          20|
| 67005410|      Feliciano|Amsterdam|          17|
|241644101|     Wittenberg|Amsterdam|          14|
|448782489|        Martijn|Rotterdam|          36|
|432320567|         Tijmen|Rotterdam|          15|
|244141635|        Culture|Rotterdam|          12|
|436284753|     Wikkelboat|Rotterdam|          11|
|302893992|        Michiel|Rotterdam|          10|
|  5285926|      Diederick| Den Haag|          29|
|  9165668|         Robert| Den Haag|          23|
|   300966|          Elwin| Den Haag|          20|
|244520390|          Gunni| Den Haag|          18|
| 10239880|          Lucas| Den Haag|          17|
+---------+---------------+---------+------------+



In [47]:
most_listings_per_city.toPandas().to_csv("gs://assignment2_airbnb/Results/most_listings_per_city.csv")

In [49]:
#get number of listings per city
query_nr_listings_per_city = """
SELECT
    listing_city.city as city,
    COUNT(*) as num_listings
FROM
    listings, listing_city
WHERE
    listings.id = listing_city.id
GROUP BY
    listing_city.city
ORDER BY
    listing_city.city   
"""

results_nr_listings_per_city = spark.sql(query_nr_listings_per_city)
results_nr_listings_per_city.show()

+---------+------------+
|     city|num_listings|
+---------+------------+
|Amsterdam|        6884|
| Den Haag|        1078|
|Rotterdam|        1041|
+---------+------------+



In [50]:
results_nr_listings_per_city.toPandas().to_csv("gs://assignment2_airbnb/Results/no_listings_per_city.csv")

In [51]:
spark.stop()